# Human Action Recognition with OpenVINO

This notebook demonstrates live human action recognition with OpenVINO. We use the [Action Recognition Models](https://docs.openvino.ai/2020.2/usergroup13.html) from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo), specifically the [Encoder](https://docs.openvino.ai/2020.2/_models_intel_action_recognition_0001_encoder_description_action_recognition_0001_encoder.html) and [Decoder](https://docs.openvino.ai/2020.2/_models_intel_action_recognition_0001_decoder_description_action_recognition_0001_decoder.html). Both models create a sequence to sequence (`"seq2seq"`)<sup id="a1">[1](#f1)</sup> system to identify the  human activities for [Kinetics-400 dataset](https://deepmind.com/research/open-source/kinetics). The models use the Video Transformer approach with ResNet34 encoder<sup id="a2">[2](#f2)</sup>. We will see in this notebook how to create the following pipeline:

<img align='center' src="action_recognition_pipeline.jpeg" alt="drawing" width="1000"/>


At the bottom of this notebook, you will see live inference results from your webcam. You can also upload a video file.

NOTE: _To use the webcam, you must run this Jupyter notebook on a computer with a webcam. If you run on a server, the webcam will not work. However, you can still do inference on a video in the final step._

<b id="f1">1</b> seq2seq: Deep learning models that take a sequence of items to the input and output. In this case, input: video frames, output: actions sequence. This `"seq2seq"` is composed of an encoder and a decoder. The encoder captures the `"context"` of the inputs to be analyzed by the decoder, and finally gets the human action and confidence.[$\hookleftarrow$](#a1)

<b id="f2">2</b>  [Video Transformer](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)#:~:text=A%20transformer%20is%20a%20deep,in%20computer%20vision%20(CV).), and [ResNet34](https://www.kaggle.com/pytorch/resnet34). [$\hookleftarrow$](#a2)



## Imports

In [ ]:
import sys
import collections
import os
import time

import cv2
import numpy as np
from IPython import display
from numpy.lib.stride_tricks import as_strided
from openvino import inference_engine as ie

from openvino.inference_engine import get_version

sys.path.append("../utils")
import notebook_utils as utils



## The models

### Download the models

We use `omz_downloader`, which is a command-line tool from the `openvino-dev` package. `omz_downloader` automatically creates a directory structure and downloads the selected model.

In this case you can use `"action-recognition-0001"` as a model name, and the system automatically downloads the two models `"action-recognition-0001-encoder"` and `"action-recognition-0001-decoder"`

If you want to download another model (`"driver-action-recognition-adas-0002-encoder"` + `"driver-action-recognition-adas-0002-decoder"`, and `"i3d-rgb-tf"`) , please change the model name. Note: Using a model outside the list can require different pre- and post-processing. 

In [ ]:
# directory where model will be downloaded
base_model_dir = "model"

# model name as named in Open Model Zoo
model_name = "action-recognition-0001"
# selected precision (FP32, FP16, FP16-INT8)
precision = "FP16-INT8"

model_path = f"model/intel/{model_name}/{model_name}/{precision}/{model_name}.xml"
model_weights_path = f"model/intel/{model_name}/{model_name}/{precision}/{model_name}.bin"

if not os.path.exists(model_path):
    download_command = f"omz_downloader " \
                       f"--name {model_name} " \
                       f"--precision {precision} " \
                       f"--output_dir {base_model_dir}"
    ! $download_command
    

### Check the model path of your models

We will be able to work with the models locally. Please check that you can see your models in your local machine after your download process.

In [ ]:
# Creating the proper path for architecture and weights model files.
model_path_decoder = f"model/intel/{model_name}/{model_name}-decoder/{precision}/{model_name}-decoder.xml"
model_weights_path_decoder = f"model/intel/{model_name}/{model_name}-decoder/{precision}/{model_name}-decoder.bin"

model_path_encoder = f"model/intel/{model_name}/{model_name}-encoder/{precision}/{model_name}-encoder.xml"
model_weights_path_encoder = f"model/intel/{model_name}/{model_name}-encoder/{precision}/{model_name}-encoder.bin"

# Testing the location of *.bin and *.xml
files = os.listdir(f"model/intel/{model_name}/{model_name}-encoder/{precision}")
print(files)

### Load your labels

We are using [Kinetics-400 dataset](https://deepmind.com/research/open-source/kinetics), and also we are providing you with the text file with the labels for this dataset.

In [ ]:
# If you want to use another model, you should change the labels file
labels = "data/kinetics.txt"

if labels:
    with open(labels) as f:
        labels = [line.strip() for line in f]
else:
    labels = None
    
#print(labels[0], np.shape(labels))

### Load the models

We will load the two models for this particular architecture, Encoder and Decoder. Downloaded models are located in a fixed structure, indicating vendor, model name, and precision.

 1. Initialize inference engine (IECore)
 2. Read the network from *.bin and *.xml files (weights and architecture)
 3. Load the model on the "CPU."
 4. Get input and output names of nodes.

Only a few lines of code are required to run the model. Let's see it.

#### Model Initialization function

In [ ]:
#Model initialization

# initialize inference engine
ie_core = ie.IECore()

def model_init(model: str, weights: str) -> dict:
    """
    Read the network and weights from file, load the 
    model on the CPU and get input and output names of nodes
    
    :param: model: model architecture path *.xml
    :param: model: model weights path *.bin
    :retuns: 
            exec_net: Encoder model network
            input_key: Input node network
            output_key: Output node network
    """
    
    # read the network and corresponding weights from file
    net = ie_core.read_network(model, weights)
    # load the model on the CPU (you can use GPU or MYRIAD as well)
    exec_net = ie_core.load_network(net, "CPU")
    # get input and output names of nodes
    input_key = list(exec_net.input_info)[0]
    output_keys = list(exec_net.outputs.keys())
    return input_key, output_keys, exec_net

#### Initialization for Encoder and Decoder

In [ ]:
#Encoder initialization 
input_key_en, output_keys_en, exec_net_en = model_init(model_path_encoder, model_weights_path_encoder)
#Decoder initialization 
input_key_de, output_keys_de, exec_net_de = model_init(model_path_decoder, model_weights_path_decoder)

# get input size - Encoder
height_en, width_en = exec_net_en.input_info[input_key_en].tensor_desc.dims[2:]
# get input size - Decoder
height_de = exec_net_de.input_info[input_key_de].tensor_desc.dims[2:]

### Helper functions

We will use the next AI functions to run the architecture Encoder-Decoder of ResNet34

1. Preprocess the input image before running the Encoder model. (`center_crop` and `adaptative_resize`)
2. Decode top-3 probabilities into label names. (`decode_output`)

Also, we can see here the minimal configuration to display text over the live streaming or video for the main process.


In [ ]:
def center_crop(frame: np.array) -> np.ndarray:
    """
    Center crop squared the original frame to standardize the input image to the encoder model
    
    :param frame: input frame
    :returns: center-crop-squared frame
    """
    img_h, img_w, _ = frame.shape
    min_dim = min(img_h, img_w)
    start_x = int((img_w - min_dim) /2.)
    start_y = int((img_h - min_dim) /2.)
    return frame[start_y:start_y+min_dim, start_x:start_x+min_dim, ...]

def adaptive_resize(frame: np.array, size: int) -> np.ndarray:
    """
    Adaptative resize of the cropped image to adjust to the input size model 224
    
    :param frame: input frame
    :param size: input size to encoder model
    :returns: resized frame, np.array type 
    """
    h, w, _ = frame.shape
    scale = size / min(h, w)
    ow, oh = int(w * scale), int(h * scale)
    if ow == w and oh == h:
        return frame
    return cv2.resize(frame, (ow, oh))

def decode_output(probs: np.array, labels: np.array, top_k: int = None) -> np.ndarray:
    """
    Decodes top probabilities into corresponding label names
    
    :param probs: confidence vector for 400 actions
    :param labels: list of actions
    :param top_k: The k most probable positions in the list of labels
    :returns: decoded_labels: The k most probable actions from the labels list
              decoded_top_probs: confidence for the k most probable actions   
    """
    top_k = 3
    #top_ind = np.argsort(probs)[::-1][:top_k]
    top_ind = np.argsort(-1*probs)[:top_k]
    #print(top_ind)
    #print(top_ind.shape)
    out_label = np.array(labels)[top_ind.astype(int)]
    decoded_labels = [out_label[0][0], out_label[0][1], out_label[0][2]]
    top_probs = np.array(probs)[0][top_ind.astype(int)]
    decoded_top_probs = [top_probs[0][0], top_probs[0][1], top_probs[0][2]]
    #print(top_probs[0][0], top_probs[0][1], top_probs[0][2])
    return decoded_labels, decoded_top_probs

"""Configuration for displaying images with text"""
FONT_COLOR = (255, 255, 255)
FONT_STYLE = cv2.FONT_HERSHEY_DUPLEX
FONT_SIZE = 0.7
TEXT_VERTICAL_INTERVAL = 25
TEXT_LEFT_MARGIN = 15    
        

### AI Pipeline

Following this pipeline, 

<img align='center' src="action_recognition_pipeline.jpeg" alt="drawing" width="1000"/>


we will use the next functions to help us to:

1. Preprocess frame before running the Encoder. (`preprocessing`)
2. Encoder Inference per frame. (`encoder`)
3. Decoder inference per set of frames. (`decoder`)
4. Normalize the Decoder output to get confidence values per action recognition label. (`softmax`)

Also, we can see here the minimal configuration to display text over the live streaming or video for the main process.

In [ ]:
def preprocessing(frame: np.ndarray, size: int) -> np.ndarray:
    """
    Preparing frame before Encoder.
    
    The image should be scaled to its shortest dimension at "size" 
    and cropped, centered, and squared so that both width and 
    height have lengths "size". Frame must be transposed from 
    Height-Width-Channels (HWC) to Channels-Height-Width (CHW).
    
    :param frame: input frame 
    :param size: input size to encoder model
    :returns: resized and cropped frame
    """
    # Adaptative resize
    preprocessed = adaptive_resize(frame, size)
    # Center_crop                
    preprocessed = center_crop(preprocessed)
    # Transpose frame HWC -> CHW
    preprocessed = preprocessed.transpose((2, 0, 1))  # HWC -> CHW
    return preprocessed

def encoder(preprocessed: np.ndarray, exec_net_en: dict, input_key_en: list, encoder_output: np.ndarray) -> np.ndarray:
    """
    Encoder Inference per frame. This function calls the network previously 
    configured for the encoder model (exec_net_en), extracts the data 
    from the output node, and appends it in an array to be used by the decoder.
    
    :param: preprocessed: preprocessing frame
    :param: exec_net_en: Encoder model network
    :param: input_key_en: Input node network
    :param: encoder_output: embedding layer that is appended with each arriving frame
    :returns: encoder_output: embedding layer that is appended with each arriving frame
    """
    # get results on action-recognition-0001-encoder model
    infer_result_encoder = exec_net_en.infer(inputs={input_key_en: preprocessed})
    encoder_result = infer_result_encoder[output_keys_en[0]]
    encoder_output.append(encoder_result)
    return encoder_output

def decoder(encoder_output: np.array, exec_net_de: dict, input_key_de: list) -> np.ndarray:
    """
    Decoder inference per set of frames. This function concatenates the embedding layer
    froms the encoder output, transpose the array to match with the decoder input size. 
    Calls the network previously configured for the decoder model (exec_net_de), extracts
    the logits and normalize those to get confidence values along specified axis.
    Decodes top probabilities into corresponding label names 
        
    :param: encoder_output: embedding layer for 16 frames
    :param: exec_net_de: Decoder model network
    :param: input_key_de: Input node network
    :returns: decoded_labels: The k most probable actions from the labels list
              decoded_top_probs: confidence for the k most probable actions
    """
    # Concatenate sample_duration frames in just one array
    decoder_input = np.concatenate(encoder_output, axis=0)
    # Organize input shape vector to the Decoder (shape: [1x16x512]]
    decoder_input = decoder_input.transpose((2, 0, 1, 3))
    decoder_input = np.squeeze(decoder_input, axis=3)

    # get results on action-recognition-0001-decoder model
    logits = exec_net_de.infer(inputs={input_key_de: decoder_input})
    result_de = logits[output_keys_de[0]]
    # Normalize logits to get confidence values along specified axis
    probs = softmax(result_de - np.max(result_de))
    # Decodes top probabilities into corresponding label names
    decoded_labels, decoded_top_probs = decode_output(probs, labels, top_k=3)
    return decoded_labels, decoded_top_probs


def softmax(x: np.array, axis: int = None) -> np.ndarray:
    """
    Normalizes logits to get confidence values along specified axis
    
    x: np.array, axis=None
    """
    exp = np.exp(x)
    return exp / np.sum(exp, axis=axis)

### Main Processing Function

Run action recognition function will run in different operations, either a webcam or a video file. See the list of procedures below:

1. Create a video player to play with target fps (`utils.VideoPlayer`).
2. Prepare a set of frames to be encoded-decoded.
3. Run AI functions
4. Visualize the results.

In [ ]:
def run_action_recognition(source: str = 0, flip: bool = False, use_popup: bool = False, skip_first_frames: int = 0):
    """
    Use the "source" webcam or video file to run the complete pipeline for action-recognition problem
    1. Create a video player to play with target fps
    2. Prepare a set of frames to be encoded-decoded
    3. Preprocess frame before Encoder
    4. Encoder Inference per frame
    5. Decoder inference per set of frames
    6. Visualize the results
    
    :param: source: webcam "0" or video path
    :param: flip: to be used by VideoPlayer function for flipping capture image
    :param: use_popup: False for showing encoded frames over this notebook, True for creating a popup window.
    :param: skip_first_frames: to be used by VideoPlayer function for skipping first frames
    :returns: display video over the notebook or in a popup window
    
    """
    
    sample_duration=16
    size=224
    #create a video player to play with target fps
    player = utils.VideoPlayer(source, flip=flip, fps=30, skip_first_frames=skip_first_frames)
    # start capturing
    player.start()
    try:
        if use_popup:
            title = "Press ESC to Exit"
            cv2.namedWindow(title, cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)
        
        processing_times = collections.deque()
        frames = []
        video_frames = []
        encoder_output = []
        decoded_labels = [0, 0, 0]
        decoded_top_probs = [0, 0, 0]
        while True:
            
            encoder_output = []
            # loop over the number of required sample frames
            # Prepare a set of frames to be enconded-decoded
            for i in range(0, sample_duration):
                # read a frame from the video stream
                frame = player.next()
                frames.append(frame)
                
                if frame is None:
                    print("Source ended")
                    break
                scale = 1280 / max(frame.shape)
                
                # Adaptative resize for visualization
                if scale < 1:
                    frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
                
                # Preprocess frame before Encoder
                preprocessed = preprocessing(frame, size)
                                
                # measure processing time
                start_time = time.time()
                
                #Encoder Inference per frame
                encoder_output = encoder(preprocessed, exec_net_en, input_key_en, encoder_output)
                
                #Decoder inference per set of frames
                # wait for sample duration to work with decoder model
                if len(encoder_output) == sample_duration:
                    decoded_labels, decoded_top_probs = decoder(encoder_output, exec_net_de, input_key_de)
                                        
                #Inference has finished ... Let's to display results
                stop_time = time.time()
                
                # calculate processing time
                processing_times.append(stop_time - start_time)
                # create a text template to show inference results over video
                text_template = '{label},{conf:.2f}%,{Time:.1f}ms,{fps:.1f}FPS'
                # use processing times from last 200 frames
                if len(processing_times) > 200:
                    processing_times.popleft()
                    
                _, f_width = frame.shape[:2]
                # mean processing time [ms]
                processing_time = np.mean(processing_times) * 1000
                fps = 1000 / processing_time

                #Visualize the results
                for i in range(0, 3):
                    display_text = text_template.format(label=decoded_labels[i], conf=decoded_top_probs[i] * 100, Time=processing_time, fps=fps)
                    text_loc = (TEXT_LEFT_MARGIN, TEXT_VERTICAL_INTERVAL * (i + 1))
                    cv2.putText(frame, display_text, text_loc, FONT_STYLE, FONT_SIZE, FONT_COLOR)
                
                # use this workaround if there is flickering
                if use_popup:
                    cv2.imshow(title, frame)
                    key = cv2.waitKey(1)
                    # escape = 27
                    if key == 27:
                        break
                else:
                    # encode numpy array to jpg
                    _, encoded_img = cv2.imencode(".jpg", frame, params=[cv2.IMWRITE_JPEG_QUALITY, 90])
                    # create IPython image
                    i = display.Image(data=encoded_img)
                    # display the image in this notebook
                    display.clear_output(wait=True)
                    display.display(i)
    # ctrl-c
    except KeyboardInterrupt:
        print("Interrupted")
    # any different error
    except RuntimeError as e:
        print(e)

    # stop capturing
    player.stop()
    if use_popup:
        cv2.destroyAllWindows()


### Run Action Recognition using your webcam

> NOTE: _To use the webcam, you must run this Jupyter notebook on a computer with a webcam. If you run on a server, the webcam will not work. However, you can still do inference on a video in the final step._

In [ ]:
run_action_recognition(source=0, flip=False, use_popup=False, skip_first_frames=0)

### Run Action Recognition on a Video File

If you don't have a webcam, you can still run this demo with a video file. Any format supported by OpenCV will work (see: https://docs.opencv.org/4.5.1/dd/d43/tutorial_py_video_display.html). You can skip first N frames to fast forward video.

In [ ]:
video_file = "C:/Intel/notebooks/openvino_notebooks/notebooks/501-action-webcam/video/paula.mp4"
#"https://github.com/intel-iot-devkit/sample-videos/raw/master/head-pose-face-detection-female.mp4?raw=true"
#"https://github.com/intel-iot-devkit/sample-videos/blob/master/store-aisle-detection.mp4?raw=true"

run_action_recognition(video_file, flip=False, use_popup=False, skip_first_frames=500)